In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Bidirectional,LSTM
from tensorflow.keras.models import load_model
from keras.layers import Input, Bidirectional, LSTM, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np
import keras
import gc 
import pandas as pd
from tqdm import tqdm

In [9]:
model_path = 'best_model.h5'
npy_path = fr"..\Data\09-12_X_test.npy"

In [10]:
A = np.load(npy_path,mmap_mode='r') #this is N_obsx100x15 
A.shape

(81600, 100, 15)

In [11]:
model = load_model(model_path)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss="categorical_crossentropy", metrics=["accuracy"])

In [14]:
def create_submission(npy_path):
    A = np.load(npy_path,mmap_mode='r') #this is N_obsx100x15 
    N = len(A)
    results = []
    
    batch_size = 10000
    obs_id =0 
    for i in tqdm(range((N + batch_size - 1) // batch_size),desc='Processing'):

        
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, N)
        batch = A[start_idx:end_idx]
        
        batch = A[i*batch_size:(i+1)*(batch_size)]
        #seq = np.expand_dims(seq, axis=0)
        predicted = model.predict(batch)

        #print(predicted)
        for j in range(len(predicted)):
            res = predicted[j].argmax()
            results.append([obs_id,res])
            obs_id  += 1

        del(batch)
        gc.collect()
        
    df_res = pd.DataFrame(results, columns=['obs_id', 'eqt_code_cat'])

    df_res.to_csv('../Data/submission-09-12-new.csv',index=False)

In [15]:
create_submission(npy_path)

Processing:   0%|          | 0/9 [00:00<?, ?it/s]

313/313 [==============================] - 4s 10ms/step


Processing:  11%|█         | 1/9 [00:03<00:31,  3.96s/it]

313/313 [==============================] - 3s 10ms/step


Processing:  22%|██▏       | 2/9 [00:07<00:25,  3.59s/it]

...

Processing:  78%|███████▊  | 7/9 [00:24<00:06,  3.38s/it]

313/313 [==============================] - 3s 10ms/step


Processing:  89%|████████▉ | 8/9 [00:27<00:03,  3.38s/it]

50/50 [==============================] - 1s 10ms/step


Processing: 100%|██████████| 9/9 [00:28<00:00,  3.15s/it]
